# 1. On cré un programme qui affiche un ecran blanc.

* Définition des paramètres de la fenètre d'affichage
* boucle principale: events et rendering

In [4]:
import pygame
import sys   

pygame.init()

##### Définir le paramètres de notre fenetre ------------------

pygame.display.set_caption("Affichage d'un écran blanc")

screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))

WHITE = (255, 255, 255)

##### Boucle principale ------------------------------------------

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    screen.fill(WHITE)

    ##### Rendering ----------------------------------------------

    pygame.display.flip()

    
pygame.quit()
sys.exit()


SystemExit: 

# 2. On implémente un personnage que nous pourrons déplacer.
* paramètres du personage
* gestion de input clavier
* gestion des events

+ paramètre du jeux: rafraichissment
+ gestion des collisions avec les bords de l'écran

* déportation du system de mouvement dans un module player.py



In [ ]:
import pygame
import sys 
from systems.player_system import move_manager 

pygame.init()

##### Définir le paramètres de notre fenetre ---------------------

pygame.display.set_caption("Affichage d'un écran blanc")

screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))

WHITE = (255, 255, 255)

##### Paramètre de notre personnage -----------------------------

# Taille et position initiale du personnage
player_width = 24
player_height = 16
player_x = (screen_width - player_width) // 2  # Centré horizontalement
player_y = (screen_height - player_height) // 2  # Centré verticalement

player_speed = 5 # Vitesse de déplacement du personnage

##### Paramètres du jeux ----------------------------------------

# clock = pygame.time.Clock()  # Définir la vitesse de rafraîchissement  # <------


##### Boucle principale ------------------------------------------
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
     
    #### Obtenir les touches pressées ----------------------------
    keys = pygame.key.get_pressed()

    #### Déplacer le personnage à gauche et à droite--------------
    if keys[pygame.K_LEFT]:
        player_x -= player_speed
    if keys[pygame.K_RIGHT]:
        player_x += player_speed

    # Empêcher le personnage de sortir de l'écran
    if player_x < 0:
        player_x = 0
    if player_x > screen_width - player_width:
        player_x = screen_width - player_width

    # player_x,_,_ = move_manager(keys, player_x, player_speed, player_width, screen_width)
    

    ##### Rendering ----------------------------------------------

    screen.fill(WHITE)  # a inverser pour le LC
    
    # Dessiner le personnage
    pygame.draw.rect(screen, (0, 128, 255), (player_x, player_y, player_width, player_height))

    # clock.tick(60)      # Limiter à 60 images par seconde   # <-----
    

    
    pygame.display.flip()

    
pygame.quit()
sys.exit()

# 3. On ajoute un fond et un sprite à notre personnage

In [2]:
import pygame
import sys   
from systems.player_system import move_manager 

pygame.init()

##### Définir le paramètres de notre fenetre ---------------------

pygame.display.set_caption("Affichage d'un écran blanc")

screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))

WHITE = (255, 255, 255)

##### Paramètre du niveau ------------------------------------

background = pygame.image.load('assets/level_11.jpg')

background_width = 6400
background_height = 1800

# Position initiale de la caméra 
camera_x = 0
camera_y = background_height - screen_height  # Placer la caméra en bas de l'image

##### Paramètre de notre personnage -----------------------------

player_width = 24
player_height = 16
player_x = (screen_width - player_width) // 2  
player_y = (screen_height - player_height) // 2

player_sprite_sheet = pygame.image.load('assets/mario_spritesheet_1.png')

# Découper les frames de la sprite sheet
frames = []
for i in range(6):  # Il y a 6 frames
    frame = player_sprite_sheet.subsurface(pygame.Rect(i * player_width, 0, player_width, player_height))
    frames.append(frame)

player_speed = 5 

# Index de la frame actuelle
current_frame = 0


##### Paramètres du jeux ----------------------------------------

clock = pygame.time.Clock()  


##### Boucle principale ------------------------------------------

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
     
    #### Obtenir les touches pressées ----------------------------
    keys = pygame.key.get_pressed()

    #### Déplacer le personnage à gauche et à droite--------------

    player_x,_,_ = move_manager(keys, player_x, player_speed, player_width, screen_width)
    

    ##### Rendering ----------------------------------------------

    screen.fill(WHITE) 

    screen.blit(background, (-camera_x, -camera_y))

    # Dessiner le personnage avec la sprite sheet
    screen.blit(frames[current_frame], (player_x, player_y))


    pygame.display.flip()

    clock.tick(60)   
        
pygame.quit()
sys.exit()

SystemExit: 

# 4 on ajoute l'animation du personnage, et un système de saut

In [4]:
import pygame
import sys
from systems.player_system import move_manager

pygame.init()

##### Définir le paramètres de notre fenetre ---------------------
pygame.display.set_caption("Workshop video_game")

screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
WHITE = (255, 255, 255)

##### Paramètre du niveau ------------------------------------

background = pygame.image.load('assets/level_11.jpg')
background_width = 6400
background_height = 1800

camera_x = 0
camera_y = background_height - screen_height 

##### Paramètre de notre personnage -----------------------------

player_width = 24
player_height = 16
player_x = screen_width // 2 - player_width // 2  # Centré horizontalement
player_y = (background_height - 110) - player_height   # Positionné au sol (bas du fond)

player_sprite_sheet = pygame.image.load('assets/mario_spritesheet_1.png')
frames = []
for i in range(6):  # Il y a 6 frames
    frame = player_sprite_sheet.subsurface(pygame.Rect(i * player_width, 0, player_width, player_height))
    frames.append(frame)

player_speed = 5

global jump_power
jump_power = -17

player_velocity_x = 0
player_velocity_y = 0


global player_state
player_state = "land"  # "land" = au sol, "air" = en l'air


current_frame = 0 

global facing_right
facing_right = False   

##### Paramètres du jeu ----------------------------------------

clock = pygame.time.Clock()

global gravity
gravity = 1 

#### Boucle principale du jeu -------------------

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    #### inputs -------------------------------------

    keys = pygame.key.get_pressed()

    #### déplacement du personnage ------------------

    player_x, player_y, player_velocity_y, facing_right, current_frame, player_state = move_manager( 
        keys, 
        player_x,
        player_y, 
        player_speed, 
        player_width,
        player_height,
        jump_power,
        gravity,  
        screen_width,
        background_height,
        facing_right, 
        current_frame,
        player_velocity_x,
        player_velocity_y,
        player_state
    )

    #### Render ------------------------------------------

    
    screen.blit(background, (-camera_x, -camera_y))

    
    adjusted_player_x = player_x - camera_x
    adjusted_player_y = player_y - camera_y + 30

    # Si le joueur fait face à droite, retourner la sprite
    if facing_right:
        flipped_frame = pygame.transform.flip(frames[current_frame], True, False)  # True = flip horizontal
        screen.blit(flipped_frame, (adjusted_player_x, adjusted_player_y))
    else:
        screen.blit(frames[current_frame], (adjusted_player_x, adjusted_player_y))

    
    pygame.display.flip()

    
    clock.tick(60)


pygame.quit()
sys.exit()

SystemExit: 

# 5. Ajout d'un scrolling horizontal avec une caméra dynamique, directement sous forme de module.

* ajout de BGM: Main theme
* ajout de SFX: Jump fx

In [2]:
import pygame
import sys
from systems.player_system import move_manager
from systems.camera_system import camera_manager
from systems.render_system import render_manager
from systems.audio_system import initialize_audio

pygame.init()

##### Définir le paramètres de notre fenetre ---------------------
pygame.display.set_caption("Workshop video_game")

screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
WHITE = (255, 255, 255)

##### Paramètre du niveau ------------------------------------

background = pygame.image.load('assets/level_11.jpg')
background_width = 6400
background_height = 1800

camera_x = 0
camera_y = background_height - screen_height 

##### Paramètre de notre personnage -----------------------------

player_width = 24
player_height = 16
player_x = screen_width // 2 - player_width // 2  # Centré horizontalement
player_y = (background_height - 100) - player_height   # Positionné au sol (bas du fond)

facing_right = False  

player_speed = 5
jump_power = -17
player_velocity_x = 0
player_velocity_y = 0

player_state = "land"  # "land" = au sol, "air" = en l'air

player_sprite_sheet = pygame.image.load('assets/mario_spritesheet_1.png')

frames = []
for i in range(6):  # Il y a 6 frames
    frame = player_sprite_sheet.subsurface(pygame.Rect(i * player_width, 0, player_width, player_height))
    frames.append(frame)

# Index de la frame actuelle
current_frame = 0  # Initialisation de l'index des frames pour éviter une erreur

##### Paramètres du jeu ----------------------------------------

sfx_bank = initialize_audio()
clock = pygame.time.Clock()
gravity = 1 

#### Boucle principale du jeu -------------------

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    #### inputs -------------------------------------

    keys = pygame.key.get_pressed()

    #### déplacement du personnage ------------------

    player_x, player_y, player_velocity_y, facing_right, current_frame, player_state = move_manager( 
        keys, 
        player_x,
        player_y, 
        player_speed, 
        player_width,
        player_height,
        jump_power,
        gravity,  
        background_width,    #   <---
        background_height,
        facing_right,
        sfx_bank, 
        current_frame,
        player_velocity_x,
        player_velocity_y,
        player_state, 
    )

    #### Camera ------------------------------------------

    camera_x, camera_y, adjusted_player_x, adjusted_player_y = camera_manager(
        player_x, 
        player_y, 
        screen_width, 
        screen_height, 
        background_width, 
        background_height
    )

    #### Render ------------------------------------------

    render_manager(background, screen, facing_right, camera_x, camera_y, frames, current_frame, adjusted_player_x, adjusted_player_y)

    clock.tick(60)


pygame.quit()
sys.exit()


SystemExit: 

# 6. creation du lvl 1.1 avec Numpy


In [ ]:
import pygame
import sys
from systems.player_system import move_manager
from systems.camera_system import camera_manager
from systems.render_system import render_manager
from systems.audio_system import initialize_audio

pygame.init()

##### Définir le paramètres de notre fenetre ---------------------
pygame.display.set_caption("Workshop video_game")

screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
WHITE = (255, 255, 255)

##### Paramètre du niveau ------------------------------------

background = pygame.image.load('assets/level_11.jpg')
background_width = 6400
background_height = 1800

camera_x = 0
camera_y = background_height - screen_height 

##### Paramètre de notre personnage -----------------------------

player_width = 24
player_height = 16
player_x = screen_width // 2 - player_width // 2  # Centré horizontalement
player_y = (background_height - 100) - player_height   # Positionné au sol (bas du fond)

facing_right = False  

player_speed = 5
jump_power = -17
player_velocity_x = 0
player_velocity_y = 0

player_state = "land"  # "land" = au sol, "air" = en l'air

player_sprite_sheet = pygame.image.load('assets/mario_spritesheet_1.png')

frames = []
for i in range(6):  # Il y a 6 frames
    frame = player_sprite_sheet.subsurface(pygame.Rect(i * player_width, 0, player_width, player_height))
    frames.append(frame)

# Index de la frame actuelle
current_frame = 0  # Initialisation de l'index des frames pour éviter une erreur

##### Paramètres du jeu ----------------------------------------

sfx_bank = initialize_audio()
clock = pygame.time.Clock()
gravity = 1 

#### Boucle principale du jeu -------------------

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    #### inputs -------------------------------------

    keys = pygame.key.get_pressed()

    #### déplacement du personnage ------------------

    player_x, player_y, player_velocity_y, facing_right, current_frame, player_state = move_manager( 
        keys, 
        player_x,
        player_y, 
        player_speed, 
        player_width,
        player_height,
        jump_power,
        gravity,  
        background_width,    #   <---
        background_height,
        facing_right,
        sfx_bank, 
        current_frame,
        player_velocity_x,
        player_velocity_y,
        player_state, 
    )

    #### Camera ------------------------------------------

    camera_x, camera_y, adjusted_player_x, adjusted_player_y = camera_manager(
        player_x, 
        player_y, 
        screen_width, 
        screen_height, 
        background_width, 
        background_height
    )

    #### Render ------------------------------------------

    render_manager(background, screen, facing_right, camera_x, camera_y, frames, current_frame, adjusted_player_x, adjusted_player_y)

    clock.tick(60)


pygame.quit()
sys.exit()